节点较多时取连通部分算

In [2]:
try:   
    distance = 1 + 1 + 1
except:
    print(1)
    distance = None

2992


In [1]:
import pandas as pd
import json
import math
import datetime
import multiprocessing as mp
import numpy as np
import pickle
import networkx as nx



import time
import datetime


# 读取数据
allFrmIndexDict = {}
idx = 0
with open('../public/data.pickle', 'rb') as f:
    networkData = pickle.load(f)
    allGraph = {}
    for logId in list(networkData.keys()):
        logData = networkData[logId]
        graphData = {}
        for frm in logData["frames"]:
            G = nx.Graph()
            G.add_nodes_from(logData[str(frm)]["nodes"])
            G.add_edges_from(logData[str(frm)]["edges"])
            graphData[str(frm)] = {'graph': G, 'egoInfos': logData[str(frm)]["ego"]}
            allFrmIndexDict[idx] = {'logId':logId, 'frame':frm}
            idx += 1
        allGraph[logId] = graphData
logList = list(networkData.keys())
print(len(logList))

47


In [2]:
# networkData["02cf0ce1-699a-373b-86c0-eb6fd5f4697a"]["frames"]

In [3]:
# graph_ = allGraph["26d141ec-f952-3908-b4cc-ae359377424e"]['0']['graph']

In [4]:
# nodes_num = len(graph_.nodes)
# nodes_num
# VEHNUM = 0
# PEONUM = 0
# coms = []
# for c in nx.connected_components(graph_):
#     if len(c)==1:
#         nodeG = graph_.subgraph(c).copy()
#         if nodeG.nodes[list(c)[0]]["label_class"]=="VEHICLE":
#             VEHNUM += 1
#         else:
#             PEONUM += 1
#     else:
#         coms.append(graph_.subgraph(c).copy())
# edgesGraph = nx.Graph()
# for g_ in coms:
#     edgesGraph = nx.union(edgesGraph, g_)
# print(VEHNUM, PEONUM, coms)

In [5]:
def get_simpleGraph(graph_):
    VEHNUM = 0
    PEONUM = 0
    coms = []
    for c in nx.connected_components(graph_):
        if len(c)==1:
            nodeG = graph_.subgraph(c).copy()
            if nodeG.nodes[list(c)[0]]["label_class"]=="VEHICLE":
                VEHNUM += 1
            elif nodeG.nodes[list(c)[0]]["label_class"]=="PEOPLE":
                PEONUM += 1
            else: # ego游离
                coms.append(graph_.subgraph(c).copy())
        else:
            coms.append(graph_.subgraph(c).copy())
    edgesGraph = nx.Graph()
    for g_ in coms:
        edgesGraph = nx.union(edgesGraph, g_)
    return VEHNUM, PEONUM, edgesGraph

In [6]:
for logId in allGraph.keys():
    for frm in allGraph[logId].keys():
        gra = allGraph[logId][frm]['graph']
        freeVEH , freePEO, edgesGraph = get_simpleGraph(gra)
        allGraph[logId][frm]["nodes_num"] = len(allGraph[logId][frm]['graph'].nodes)
        allGraph[logId][frm]['freeVeh'] = freeVEH
        allGraph[logId][frm]['freePeo'] = freePEO
        allGraph[logId][frm]['edgesGraph'] = edgesGraph
    

In [7]:
same_frame_df = pd.read_csv("../public/procressData/same_frame_relationship.csv")
print(len(same_frame_df.logId.unique()))
same_frame_df

,Keys,same_frame,logId,frame
0,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 0",0,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,0
1,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 1",1,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,1
2,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 2",1,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,2
3,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 3",1,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,3
4,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 4",1,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,4
...,...,...,...,...
8167,"'ff78e1a3-6deb-34a4-9a1f-b85e34980f06', 151",150,ff78e1a3-6deb-34a4-9a1f-b85e34980f06,151
8168,"'ff78e1a3-6deb-34a4-9a1f-b85e34980f06', 152",150,ff78e1a3-6deb-34a4-9a1f-b85e34980f06,152
8169,"'ff78e1a3-6deb-34a4-9a1f-b85e34980f06', 153",150,ff78e1a3-6deb-34a4-9a1f-b85e34980f06,153
8170,"'ff78e1a3-6deb-34a4-9a1f-b85e34980f06', 154",150,ff78e1a3-6deb-34a4-9a1f-b85e34980f06,154


47

In [10]:
uniqueFrame = same_frame_df.drop_duplicates(subset=["same_frame", "logId"], keep='first')
uniqueFrame.reset_index(drop=True, inplace= True)
print(len(uniqueFrame))
unique_frame_dict = uniqueFrame[["same_frame", "logId"]].to_dict(orient="index")
uniqueFrame.iloc[:25]


3214


,Keys,same_frame,logId,frame
0,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 0",0,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,0
1,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 1",1,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,1
2,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 32",32,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,32
3,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 49",49,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,49
4,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 59",59,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,59
5,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 65",65,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,65
6,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 72",72,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,72
7,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 84",84,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,84
8,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 87",87,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,87
9,"'02cf0ce1-699a-373b-86c0-eb6fd5f4697a', 88",88,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,88


In [43]:
# 给3000多帧分配一个index 存为json后Index变成了str需注意！！！！！！！！！！！！！！！##################################

In [41]:
# import json
# jsonWriter = open("../public/procressData/index_frame_dict.json", 'w')
# jsonWriter.write(json.dumps(unique_frame_dict))
# jsonWriter.close()

In [8]:
unique_frame_dict[0]

{'same_frame': 0, 'logId': '02cf0ce1-699a-373b-86c0-eb6fd5f4697a'}

In [12]:
from tqdm import tqdm
import multiprocessing as mp

In [19]:
resDict = {}

In [13]:
def get_distances(source_index, allGraph, unique_frame_dict):
    distance_dict = {}
    frame_dictA = unique_frame_dict[source_index]
    graph_dictA = allGraph[frame_dictA["logId"]][str(frame_dictA["same_frame"])]
    for target_index in range(len(unique_frame_dict)):
        frame_dictB = unique_frame_dict[target_index]
        graph_dictB = allGraph[frame_dictB["logId"]][str(frame_dictB["same_frame"])]
        distance = nx.graph_edit_distance(graph_dictA["edgesGraph"], graph_dictB["edgesGraph"],  roots = ("ego", "ego"),  timeout = 2) + \
    abs(graph_dictA['freeVeh'] - graph_dictB['freeVeh']) + abs(graph_dictA['freePeo'] - graph_dictB['freePeo'])
        distance_dict[target_index] = distance
    return {source_index: distance_dict}

In [14]:
num_cores = int(mp.cpu_count())
        
pool = mp.Pool(num_cores)

In [15]:
index_start = 0
index_end = 32

In [ ]:
index_start = 32
index_end = 128

In [16]:
results = [pool.apply_async(get_distances, args=(source_index_, allGraph, unique_frame_dict)) for source_index_ in range(index_start, index_end)]
results = [p.get() for p in results]

In [ ]:
jsonWriter = open("../public/procressData/distance/"+index_start+"_" + index_end+".json", 'w')
jsonWriter.write(json.dumps(results))
jsonWriter.close()

In [35]:
for i in range(1):
    frame_dictA = unique_frame_dict[i]
    graph_dictA = allGraph[frame_dictA["logId"]][str(frame_dictA["same_frame"])]
    for j in tqdm(range(500)):
        frame_dictB = unique_frame_dict[j]
        graph_dictB = allGraph[frame_dictB["logId"]][str(frame_dictB["same_frame"])]
        distance = nx.graph_edit_distance(graph_dictA["edgesGraph"], graph_dictB["edgesGraph"],  roots = ("ego", "ego"),  timeout = 2) + \
    abs(graph_dictA['freeVeh'] - graph_dictB['freeVeh']) + abs(graph_dictA['freePeo'] - graph_dictB['freePeo'])
        resDict[i, j] = distance

100%|██████████| 500/500 [02:23<00:00,  3.48it/s]


In [34]:
allGraph[frame_dictB["logId"]][str(frame_dictB["same_frame"])]["edgesGraph"].nodes

NodeView(('ego', '20001015', 'dadaddd9-90cb-452c-a509-1e9bfeecb24a', '20001014', '8748e228-b500-413c-9a2a-361c4d9bb324', '09c74378-9527-4842-814e-9df7ea97a286', 'f309a431-c6aa-4e24-97a5-73a0f3ac12b6', '7c058005-603f-4cf8-9846-4e8f05071532', '20001628'))

In [21]:
j

447

In [18]:
nx.graph_edit_distance(graph_dictA["edgesGraph"], graph_dictB["edgesGraph"],  roots = ("ego", "ego"),  timeout = 2) + \
    abs(graph_dictA['freeVeh'] - graph_dictB['freeVeh']) + abs(graph_dictA['freePeo'] - graph_dictB['freePeo'])

38.0

In [14]:
graph_dictB
# allGraph["02cf0ce1-699a-373b-86c0-eb6fd5f4697a"].keys()

{'graph': <networkx.classes.graph.Graph at 0x196546335e0>,
 'egoInfos': ('ego',
  {'label_class': 'ego',
   'speed': 22.011210315302463,
   'acceleration': 1.5262151701431392,
   'yawDiff': 0.023420526610522}),
 'nodes_num': 20,
 'freeVeh': 17,
 'freePeo': 1,
 'edgesGraph': <networkx.classes.graph.Graph at 0x1965ac91df0>}

In [17]:
graph_dictA["edgesGraph"].edges

EdgeView([('ego', '20001832'), ('202d03cb-5d63-4778-99ae-b8dce3131f56', '20001615'), ('90fd0e09-9f50-4851-9fe6-39e86bf97910', '20001615'), ('7c256186-f727-4a2c-87fd-c57782364f8d', '20001173'), ('b54f4151-e692-4f42-8803-842e25e91495', '20001615'), ('f632b2f0-f72c-4d7a-9f7f-b7e70a0b1cb3', '20001832'), ('f14cbbe5-81de-4c80-ba6c-3b6ba593cf34', '20001834'), ('ed5d6f51-652d-4a28-8df2-e79c4fba8ceb', '20001832'), ('20001615', '20001832'), ('20001615', '20001834'), ('20001173', '20001834')])

In [38]:
d1 = allGraph["e9bb51af-1112-34c2-be3e-7ebe826649b4"]['1']

In [40]:
d2 = allGraph["e9bb51af-1112-34c2-be3e-7ebe826649b4"]['2']

In [47]:
nx.graph_edit_distance(d1["edgesGraph"], d2["edgesGraph"],  roots = ("ego", "ego"))

0.0

In [46]:
5434 * 5434 * 1 / 60 / 60 /10

820.2321111111111

In [ ]:
[g1.subgraph(c).copy() for c in nx.connected_components(g1) if len(c)>1]